In [1]:
import requests
import pandas as pd
import numpy as np
import ast

from datetime import datetime, timedelta
pd.options.display.max_columns = None

Usando el csv de "attacks_limpieza_completa y el csv con los datos climáticos del anterior pair, deberemos juntarlos en una única fuente de información.



In [2]:
df_tiburones = pd.read_csv("datos/attacks_limpieza_completa.csv", index_col=0)

df_clima = pd.read_csv("datos/tiempo_paises_1.csv", index_col=0)

In [3]:
df_clima.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,3,8,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",4,15,10,1015,none,0,0,235,2,USA
1,6,6,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",4,15,8,1014,none,0,0,180,3,USA


In [4]:
df_tiburones.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,country2,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,4,7,1,9.116327
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,3,7,7,3.804860


El objetivo de la sesión de hoy será juntar en un único csv la información de ambas fuentes. 
Para ello:
- Cargaremos los dos ficheros de datos
- Del dataframe de los ataques nos quedaremos solo con las filas de los países que seleccionamos en la lección de ayer: (USA, Australia, New Zealand, South Africa, Papua New Guinea)
- Del dataframe de los datos climáticos seleccionaremos todas las columnas.
- Cuando ya tengamos todos los datos deseados juntaremos los dos csv.
- Para hacer esta unión tendremos que hacer un groupby en la tabla de clima para sacar una media de las medidas climáticas por país.
- Antes de hacer el groupby si nos fijamos tenemos dos columnas rh_profile y wind_profile cuya información es una lista de diccionarios. 
Si intentamos hacer la media de eso no nos dará un valor real. A este problema ya nos enfrentamos en la clase invertida de ETL-2, donde teníais un Bonus para desempaquetar esta información. En caso de que en aquel ejercicio no lo consigierais os dejamos por aquí una posible solución que nos permite separar esa información en distintas columnas. Os dejamos el código documentado. ⚠️ Os recomendamos que vayáis desgranando el código y viendo lo que nos devuelve cada línea de código para entenderlo mejor.

In [5]:
paises = ["usa", "australia", "new zealand", "south africa", "papua new guinea"]

In [6]:
df_tiburones_country =df_tiburones[df_tiburones["country"].isin(paises)]
df_tiburones_country["country"].unique()

array(['usa', 'australia', 'south africa', 'new zealand',
       'papua new guinea'], dtype=object)

In [7]:
# Como los datos de las columnas, respecto a los paises, en un df está en capitalize y en el otro en minúsculas vamos a unificarlo un poco.

df_clima["country"] = df_clima["country"].str.lower()

In [8]:
df_clima.head(1)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,3,8,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",4,15,10,1015,none,0,0,235,2,usa


In [9]:
df_tiburones_country.head(1)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,country2,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,4,7,1,9.116327


Procedemos a limpiar las columnas del df_clima de "rh_profile" y "wind_profile" 

In [10]:
df_clima['rh_profile'] = df_clima['rh_profile'].apply(ast.literal_eval)

In [11]:
df_rh = df_clima["rh_profile"].apply(pd.Series)
df_rh.head(1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'rh': -9999}","{'layer': '900mb', 'rh': 3}","{'layer': '850mb', 'rh': 2}","{'layer': '800mb', 'rh': 2}","{'layer': '750mb', 'rh': 3}","{'layer': '700mb', 'rh': 5}","{'layer': '650mb', 'rh': 4}","{'layer': '600mb', 'rh': 2}","{'layer': '550mb', 'rh': 11}","{'layer': '500mb', 'rh': 14}","{'layer': '450mb', 'rh': 9}","{'layer': '400mb', 'rh': 6}","{'layer': '350mb', 'rh': 8}","{'layer': '300mb', 'rh': 13}","{'layer': '250mb', 'rh': 11}","{'layer': '200mb', 'rh': 3}"


In [12]:
for i in range(len(df_rh.columns)):
    
    #print(df_rh_copi[0].apply(pd.Series))
    
    nombre = "rh_" + str(df_rh[i].apply(pd.Series)["layer"][0]) 

    #print(nombre)

    valores = list(df_rh[i].apply(pd.Series)["rh"])

    #print(valores)

    df_clima.insert(i, nombre[0:14], valores)
    


In [20]:
#Hacer un group by por df clima por country y luego hacemos la media

In [13]:
df_clima.head(1)

,rh_0 950mb\n,rh_0 900mb\n,rh_0 850mb\n,rh_0 800mb\n,rh_0 750mb\n,rh_0 700mb\n,rh_0 650mb\n,rh_0 600mb\n,rh_0 550mb\n,rh_0 500mb\n,rh_0 450mb\n,rh_0 400mb\n,rh_0 350mb\n,rh_0 300mb\n,rh_0 250mb\n,rh_0 200mb\n,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,-9999,3,2,2,3,5,4,2,11,14,9,6,8,13,11,3,3,8,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",4,15,10,1015,none,0,0,235,2,usa


Hacemos lo mismo con la columna de wind_profile:

In [14]:
df_clima['wind_profile']= df_clima['wind_profile'].apply(ast.literal_eval)

In [15]:
df_wind= df_clima["wind_profile"].apply(pd.Series)
df_wind.head(1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'direction': -9999, 'speed'...","{'layer': '900mb', 'direction': 305, 'speed': 3}","{'layer': '850mb', 'direction': 295, 'speed': 3}","{'layer': '800mb', 'direction': 275, 'speed': 5}","{'layer': '750mb', 'direction': 285, 'speed': 5}","{'layer': '700mb', 'direction': 300, 'speed': 6}","{'layer': '650mb', 'direction': 285, 'speed': 6}","{'layer': '600mb', 'direction': 275, 'speed': 5}","{'layer': '550mb', 'direction': 285, 'speed': 6}","{'layer': '500mb', 'direction': 295, 'speed': 7}","{'layer': '450mb', 'direction': 295, 'speed': 7}","{'layer': '400mb', 'direction': 305, 'speed': 6}","{'layer': '350mb', 'direction': 320, 'speed': 9}","{'layer': '300mb', 'direction': 315, 'speed': 10}","{'layer': '250mb', 'direction': 315, 'speed': 12}","{'layer': '200mb', 'direction': 315, 'speed': 7}"


In [16]:
for i in range(len(df_wind.columns)):

    nombre = "wind_" + str(df_wind[i].apply(pd.Series)["layer"][0]) + "direction"
    nombre2 = "wind_" + str(df_wind[i].apply(pd.Series)["layer"][0]) + "speed"
    #print(nombre)

    valor1 = list(df_wind[i].apply(pd.Series)["direction"])
    valor2 = list(df_wind[i].apply(pd.Series)["speed"])

    #print(valores)
    
    df_clima.insert(i, nombre[:14], valor1)
    df_clima.insert(i, nombre2[:14], valor2)

In [ ]:
#dos for uno para la direccion y otro para la velocidad

In [22]:
df_wind.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'direction': -9999, 'speed'...","{'layer': '900mb', 'direction': 305, 'speed': 3}","{'layer': '850mb', 'direction': 295, 'speed': 3}","{'layer': '800mb', 'direction': 275, 'speed': 5}","{'layer': '750mb', 'direction': 285, 'speed': 5}","{'layer': '700mb', 'direction': 300, 'speed': 6}","{'layer': '650mb', 'direction': 285, 'speed': 6}","{'layer': '600mb', 'direction': 275, 'speed': 5}","{'layer': '550mb', 'direction': 285, 'speed': 6}","{'layer': '500mb', 'direction': 295, 'speed': 7}","{'layer': '450mb', 'direction': 295, 'speed': 7}","{'layer': '400mb', 'direction': 305, 'speed': 6}","{'layer': '350mb', 'direction': 320, 'speed': 9}","{'layer': '300mb', 'direction': 315, 'speed': 10}","{'layer': '250mb', 'direction': 315, 'speed': 12}","{'layer': '200mb', 'direction': 315, 'speed': 7}"
1,"{'layer': '950mb', 'direction': -9999, 'speed'...","{'layer': '900mb', 'direction': 195, 'speed': 3}","{'layer': '850mb', 'direction': 205, 'speed': 4}","{'layer': '800mb', 'direction': 220, 'speed': 4}","{'layer': '750mb', 'direction': 235, 'speed': 4}","{'layer': '700mb', 'direction': 250, 'speed': 5}","{'layer': '650mb', 'direction': 260, 'speed': 5}","{'layer': '600mb', 'direction': 300, 'speed': 6}","{'layer': '550mb', 'direction': 295, 'speed': 6}","{'layer': '500mb', 'direction': 285, 'speed': 6}","{'layer': '450mb', 'direction': 290, 'speed': 6}","{'layer': '400mb', 'direction': 295, 'speed': 6}","{'layer': '350mb', 'direction': 300, 'speed': 7}","{'layer': '300mb', 'direction': 300, 'speed': 9}","{'layer': '250mb', 'direction': 310, 'speed': 10}","{'layer': '200mb', 'direction': 315, 'speed': 7}"


In [37]:
# HEcho por ANA

for i in range(len(df_wind.columns)):

    desempaquetar = df_wind[i].apply(pd.Series)["layer"]

    print(desempaquetar)
    print("-----")    

    layer  direction  speed
0   950mb      -9999  -9999
1   950mb      -9999  -9999
2   950mb      -9999  -9999
3   950mb      -9999  -9999
4   950mb      -9999  -9999
..    ...        ...    ...
59  950mb      -9999  -9999
60  950mb      -9999  -9999
61  950mb      -9999  -9999
62  950mb      -9999  -9999
63  950mb      -9999  -9999

[320 rows x 3 columns]
-----
    layer  direction  speed
0   900mb        305      3
1   900mb        195      3
2   900mb        185      4
3   900mb        200      5
4   900mb        200      3
..    ...        ...    ...
59  900mb      -9999  -9999
60  900mb      -9999  -9999
61  900mb      -9999  -9999
62  900mb      -9999  -9999
63  900mb      -9999  -9999

[320 rows x 3 columns]
-----
    layer  direction  speed
0   850mb        295      3
1   850mb        205      4
2   850mb        200      5
3   850mb        205      5
4   850mb        200      4
..    ...        ...    ...
59  850mb      -9999  -9999
60  850mb      -9999  -9999
61  850mb      -

In [32]:
str(df_wind[0].apply(pd.Series)["layer"][0])[0:14]

#Hecho por Ana

'0    950mb\n0  '

In [35]:
list(df_wind[0].apply(pd.Series)["direction"])
#Hecho por Ana

[-9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 95,
 355,
 60,
 155,
 115,
 55,
 40,
 75,
 95,
 65,
 180,
 140,
 90,
 40,
 5,
 315,
 95,
 105,
 160,
 115,
 275,
 250,
 195,
 180,
 100,
 115,
 110,
 80,
 65,
 75,
 90,
 75,
 105,
 110,
 95,
 65,
 70,
 50,
 45,
 80,
 80,
 70,
 85,
 115,
 80,
 10,
 5,
 45,
 70,
 60,
 35,
 25,
 35,
 5,
 10,
 70,
 65,
 55,
 60,
 75,
 95,
 85,
 45,
 50,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,
 -9999,


In [17]:
df_clima.head(2)

,wind_0 950m,wind_0 900m,wind_0 850m,wind_0 800m,wind_0 750m,wind_0 700m,wind_0 650m,wind_0 600m,wind_0 550m,wind_0 500m,wind_0 450m,wind_0 400m,wind_0 350m,wind_0 300m,wind_0 250m,wind_0 200m,rh_0 950mb\n,rh_0 900mb\n,rh_0 850mb\n,rh_0 800mb\n,rh_0 750mb\n,rh_0 700mb\n,rh_0 650mb\n,rh_0 600mb\n,rh_0 550mb\n,rh_0 500mb\n,rh_0 450mb\n,rh_0 400mb\n,rh_0 350mb\n,rh_0 300mb\n,rh_0 250mb\n,rh_0 200mb\n,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,-9999,3,2,2,3,5,4,2,11,14,9,6,8,13,11,3,3,8,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",4,15,10,1015,none,0,0,235,2,usa
1,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,-9999,2,2,2,1,3,4,12,7,3,6,7,10,11,7,7,6,6,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",4,15,8,1014,none,0,0,180,3,usa


Procedemos a unir los 2 DataFrames en uno.

In [18]:
#df_total = df_tiburones_country.merge(df_clima, on="country")
#df_total.head(2)

Guardar los resultados obtenidos en un csv que usaremos en próximos ejercicios de pair programming.